<a href="https://colab.research.google.com/github/edms88/Studies/blob/main/Atendimento_CSC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import csv 
import os
import shutil
import datetime as dt 

cs00 = pd.read_csv('/content/drive/Othercomputers/Meu laptop/Braskem CSV/sn_customerservice_case.csv', encoding='ISO=8859-1', sep=',')  #pt01 importação da base

#renomeação de colunas
cs00.columns = ['Chamado','Estado','Primario','Categoria','Tipo de Servicos','Empresa','Solicitante','Aberto','Grupo de atribuição','Atribuido','Fornecedor','Cliente',\
    'Integrante','Canal','Prazo','Encerrado','Sla Total','SLA Previsto','SLA Tempo medio','Atraso']

ritm = pd.read_csv('/content/drive/Othercomputers/Meu laptop/Braskem CSV/sc_req_item.csv', encoding='ISO=8859-1', sep=',')
#renomeação de colunas
ritm.columns = ['Chamado','Estado','Primario','Categoria','Tipo de Servicos','Empresa','Solicitante','Aberto','Grupo de atribuição','Atribuido','Fornecedor','Cliente',\
    'Integrante','Canal','Prazo','Encerrado','Sla Total','SLA Previsto','SLA Tempo medio','Atraso']

#Concatenar pt1 & pt2 trocando nomes dos items de serviço
base = pd.concat([ritm,cs00], ignore_index=True).replace(['Processo da Compras ao Pagamento (Fornecedor)','Processo da Venda ao Recebimento (Cliente)',\
    'Alteração de Dados Bancários de Fornecedor','Cancelamento de Nota Fiscal - Material','Cancelamento de Nota Fiscal - Serviços',\
        'Carta/Declaração de Anuência - Nota Fiscal de Serviço','Declaração de não aproveitamento de crédito ICMS/IPI - Fornecedor',\
            'Envio de documentos para pagamento - Auxílio Conectividade','Envio de Documentos para Pagamento - Auxílio Educação',\
                'Identificação de pagamento/Abatimento','Prestação do Serviço em Desacordo (Exclusivo CT-e)',\
                    'Solicitação de Informe de Rendimentos','Portal de serviços'],['Fornecedor','Cliente','Alteração de Dados de Fornecedor','Cancelamento de NFM',\
                        'Cancelamento de NFS','Carta de Anuência','Declaração de não aproveitamento de ICMS/IPI',\
                            'Auxílio Conectividade','Auxílio Educação','Identificação de pagamento','Prestação do Serviço em Desacordo',\
                                'Informe de Rendimentos','Web']).drop(['Estado','Fornecedor','Cliente','Integrante','Canal','Sla Total','SLA Previsto','SLA Tempo medio'], axis=1)

base['Solicitante'] = base['Solicitante'].fillna('Fornecedor')  #Preenchendo valores NA
base['Empresa'] = base['Empresa'].fillna('INTEGRANTE')      #Preenchando valores NA
base = base.reindex(columns=['Chamado','Primario','Tipo de Servicos','Categoria','Empresa','Solicitante','Grupo de atribuição','Atribuido','Aberto','Encerrado','Prazo','Atraso'])  #Organizando a Dataframe

#Formatando colunas

base['Aberto'] = pd.to_datetime(base['Aberto']) 
base['Encerrado'] = pd.to_datetime(base['Encerrado'])
base['Prazo'] = pd.to_datetime(base['Prazo'])
base['ETL'] = base['Encerrado'] - base['Aberto']

base['out date'] = base['Atraso'].map({True:'Fora do prazo',False:'Dentro do prazo'})

#Exportando para excel 
base.to_excel('Base_chamado.xlsx', index=False)

#Analise e de backlogs para montar no PWBI
base['Mes Abertura'] = base['Aberto'].dt.strftime('%m')
base['Mes Encerrado'] = base['Encerrado'].dt.strftime('%m')
bl = base[['Tipo de Servicos','Mes Abertura','Mes Encerrado']]
bl1 = bl.groupby(['Mes Abertura','Mes Encerrado']).count().reset_index()
bl1 = bl1.replace(['01','02','03','04','05','06','07','08','09','10','11','12'],['01/01/2022','01/02/2022','01/03/2022','01/04/2022','01/05/2022','01/06/2022','01/07/2022','01/08/2022','01/09/2022','01/10/2022','01/11/2022','01/12/2022'])


#bl1.to_excel('backlogs.xlsx', index=False)
#shutil.move('/content/backlogs.xlsx','/content/drive/Othercomputers/Meu laptop/Braskem CSV/backlogs.xlsx')


#Analise de Tempo medio de trabalho ETL

basetm = base[['Chamado','Tipo de Servicos','Aberto','Encerrado','Prazo','ETL']]
tm = basetm.groupby(['Tipo de Servicos']).agg({'ETL':'mean','Tipo de Servicos': 'count'}).sort_values(by='ETL', ascending=False)
#tm.to_excel('Tempo medio de chamados.xlsx')

cs00task = pd.read_csv('/content/drive/Othercomputers/Meu laptop/Braskem CSV/sn_customerservice_task_case_sla.csv', encoding='ISO=8859-1', sep=',')  #importando a base de Tarefas p1

cs00task.columns = ['Tarefa','Estado','Descrição','Chamado','Tipo de Servicos','Empresa','Aberto','Grupo de Atribuicao','Atribuido','Canal','Hora de violacao','Prazo','Encerrado','Definicao Prazo','Destino','Duração','% Tempo decorrido','Tempo Útil decorrido','Tempo Restante','SLA Total Previsto','Atraso','Hora de inicio']

ritmtask = pd.read_csv('/content/drive/Othercomputers/Meu laptop/Braskem CSV/sn_customerservice_task_req_sla.csv', encoding='ISO=8859-1', sep=',') #importando a base de tarefas p2

ritmtask.columns = ['Tarefa','Estado','Descrição','Chamado','Tipo de Servicos','Empresa','Aberto','Grupo de Atribuicao','Atribuido','Canal','Hora de violacao','Prazo','Encerrado','Definicao Prazo','Destino','Duração','% Tempo decorrido','Tempo Útil decorrido','Tempo Restante','SLA Total Previsto','Atraso','Hora de inicio']


#Unificando o dateframe substituindo alguns nome de tipo de serviço
base_task = pd.concat([cs00task,ritmtask], ignore_index=True).replace(['Processo da Compras ao Pagamento (Fornecedor)','Processo da Venda ao Recebimento (Cliente)',\
    'Alteração de Dados Bancários de Fornecedor','Cancelamento de Nota Fiscal - Material','Cancelamento de Nota Fiscal - Serviços',\
        'Carta/Declaração de Anuência - Nota Fiscal de Serviço','Declaração de não aproveitamento de crédito ICMS/IPI - Fornecedor',\
            'Envio de documentos para pagamento - Auxílio Conectividade','Envio de Documentos para Pagamento - Auxílio Educação',\
                'Identificação de pagamento/Abatimento','Prestação do Serviço em Desacordo (Exclusivo CT-e)',\
                    'Solicitação de Informe de Rendimentos','Portal de serviços','SC - N2 (UNIDADES)','SC-N2 (BA)','SC-N2 (SP)','SC-N2 (RJ)','SC-N2 (ABC)','SC-N2 (AL)','SC-N2 (RS)'],['Fornecedor','Cliente','Alteração de Dados de Fornecedor','Cancelamento de NFM',\
                        'Cancelamento de NFS','Carta de Anuência','Declaração de não aproveitamento de ICMS/IPI',\
                            'Auxílio Conectividade','Auxílio Educação','Identificação de pagamento','Prestação do Serviço em Desacordo',\
                                'Informe de Rendimentos','Web','SC - N1 - ADM DE PESSOAS','SC - N1 - ADM DE PESSOAS','SC - N1 - ADM DE PESSOAS','SC - N1 - ADM DE PESSOAS','SC - N1 - ADM DE PESSOAS','SC - N1 - ADM DE PESSOAS','SC - N1 - ADM DE PESSOAS'])


base_task['Empresa'] = base_task['Empresa'].fillna('INTEGRANTE') #preenchendo dados NA
base_task['Atribuido'] = base_task['Atribuido'].fillna('INTEGRANTE') #preenchendo dados NA

# Divisão para descobrir a HORA e convertendo para formato INT
base_task['Duração'] = (base_task['Duração'] / 3600).round().astype(int) 
base_task['Tempo Restante'] = (base_task['Tempo Restante'] / 3600).round().astype(int)
base_task['SLA Total Previsto'] = (base_task['SLA Total Previsto'] / 3600).round().astype(int)

base_task = base_task.drop(['Estado','Descrição','Definicao Prazo'], axis=1) #Retirada de colunas não utilizadas

#Convertendo para STR para depois chamar como %
base_task['% Tempo decorrido'] = (base_task['% Tempo decorrido'].str.replace(',','').astype(float)) / 100

#Convertendo a formato data
base_task['Aberto'] = pd.to_datetime(base_task['Aberto'])
base_task['Hora de violacao'] = pd.to_datetime(base_task['Hora de violacao'])
base_task['Prazo'] = pd.to_datetime(base_task['Prazo'])
base_task['Encerrado'] = pd.to_datetime(base_task['Encerrado'])
base_task['Hora de inicio'] = pd.to_datetime(base_task['Hora de inicio'])
base_task['Tempo Útil decorrido'] = (base_task['Tempo Útil decorrido'] / 3600).round().astype(int)

base_task['out date'] = base_task['Atraso'].map({True:'Fora do prazo',False:'Dentro do prazo'})
#exportando como base excel
base_task.to_excel('Base tarefa.xlsx', index=False)


In [ ]:
display(base_task)

In [ ]:

# movendo as bases antigas e limpando a pasta
shutil.move('/content/Base_chamado.xlsx','/content/drive/Othercomputers/Meu laptop/Braskem CSV/Base chamado.xlsx')

shutil.move('/content/drive/Othercomputers/Meu laptop/Braskem CSV/sn_customerservice_case.csv','/content/drive/Othercomputers/Meu laptop/Braskem CSV/base antigas/sn_customerservice_case.csv')
shutil.move('/content/drive/Othercomputers/Meu laptop/Braskem CSV/sc_req_item.csv','/content/drive/Othercomputers/Meu laptop/Braskem CSV/base antigas/sc_req_item.csv')


shutil.move('/content/Base tarefa.xlsx','/content/drive/Othercomputers/Meu laptop/Braskem CSV/Base tarefa.xlsx')
shutil.move('/content/drive/Othercomputers/Meu laptop/Braskem CSV/sn_customerservice_task_case_sla.csv','/content/drive/Othercomputers/Meu laptop/Braskem CSV/base antigas/sn_customerservice_task_case_sla.csv')
shutil.move('/content/drive/Othercomputers/Meu laptop/Braskem CSV/sn_customerservice_task_req_sla.csv','/content/drive/Othercomputers/Meu laptop/Braskem CSV/base antigas/sn_customerservice_task_req_sla.csv')
#shutil.move('/content/resolução tarefas.xlsx','/content/drive/Othercomputers/Meu laptop/Braskem CSV/Resolução tarefas.xlsx')


'/content/drive/Othercomputers/Meu laptop/Braskem CSV/base antigas/sn_customerservice_task_req_sla.csv'